# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import wordninja
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Attention
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from transformer_build import  *
from data_preprocessing import *
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

In [ ]:
textcuboid=np.load('./1-Channel textcuboid_ag(bert).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_ag(bert).npy')

# 전처리

In [3]:
train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [4]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:16000]
y_test=label[16000:]

In [5]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 176

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [ ]:
tmp = [[x,y,z] for x, y, z in zip(xtext_train, textcuboid, y_train)]
random.shuffle(tmp)
xtext_train = [n[0] for n in tmp]
textcuboid = [n[1] for n in tmp]
y_train = [n[2] for n in tmp]
xtext_train=np.array(xtext_train)
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [ ]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_val=y_train[:1000]
y_train=y_train[1000:]

text_test=xtext_test
y_test=np.array(y_test)

# 모델 빌드

In [23]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [6]:
#트랜스포머 블록 적층
embedding_dim = 384  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1

#BERT channel의 학습
ELMo_input = Input(shape=(46, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(ELMo_input)
pooling = GlobalMaxPooling1D()(conv1)
ELMo_x = Dense(256, activation='relu')(pooling)
ELMo_attention_layer = Attention()
ELMo_a=ELMo_attention_layer([ELMo_x,ELMo_x])

#Sequence channel의 학습
Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    Text_x = transformer_block(Text_x)

Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)
Text_attention_layer = Attention()
Text_a=Text_attention_layer([Text_x,Text_x])

#결합 후 분류
x = layers.concatenate([ELMo_x,ELMo_a, Text_x,Text_a], axis=-1)
x = Dropout(0.5)(x)
output_layer = Dense(4, activation='softmax')(x)

model = Model(inputs=[ELMo_input, Text_inputs], outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 176)]        0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 176, 384)    15054720    ['input_2[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 input_1 (InputLayer)           [(None, 46, 768)]    0           []                               
                                                                                                  
 transformer_block (Transformer  (None, 176, 384)    617888      ['token_and_position_embeddin

In [68]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=50,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/50
59/59 [==============================] - 13s 173ms/step - loss: 2.1194 - accuracy: 0.5854 - val_loss: 0.4558 - val_accuracy: 0.8540 - lr: 0.0010
Epoch 2/50
59/59 [==============================] - 10s 162ms/step - loss: 0.5764 - accuracy: 0.8178 - val_loss: 0.4300 - val_accuracy: 0.8500 - lr: 0.0010
Epoch 3/50
59/59 [==============================] - 9s 159ms/step - loss: 0.4633 - accuracy: 0.8549 - val_loss: 0.3833 - val_accuracy: 0.8730 - lr: 0.0010
Epoch 4/50
59/59 [==============================] - 9s 159ms/step - loss: 0.4205 - accuracy: 0.8718 - val_loss: 0.3380 - val_accuracy: 0.8890 - lr: 0.0010
Epoch 5/50
59/59 [==============================] - 10s 164ms/step - loss: 0.3886 - accuracy: 0.8803 - val_loss: 0.3410 - val_accuracy: 0.8860 - lr: 0.0010
Epoch 6/50
59/59 [==============================] - 9s 160ms/step - loss: 0.4761 - accuracy: 0.8371 - val_loss: 0.3474 - val_accuracy: 0.8900 - lr: 0.0010
Epoch 7/50
59/59 [==============================] - 9s 159ms/step -

In [69]:
model.evaluate([textcuboid_test,text_test],y_test)

125/125 [==============================] - 2s 12ms/step - loss: 0.1909 - accuracy: 0.9452


[0.19094263017177582, 0.9452499747276306]

In [71]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_40.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)

125/125 [==============================] - 1s 8ms/step - loss: 0.1888 - accuracy: 0.9398


[0.1888122260570526, 0.9397500157356262]